In [1]:
## train classifier
import os
import torch
os.environ["CUDA_VISIBLE_DEVICES"]="7"
print('Num. of cuda visible devices: %d' %torch.cuda.device_count(), list(range(torch.cuda.device_count())))  

Num. of cuda visible devices: 1 [0]


In [2]:
class Options():
    def __init__(self):

        self.batch_size= 32
        self.beta1= 0.9
        self.beta2= 0.999
        self.checkpoint= None
        self.checkpoint_dir= 'checkpoints'
        self.data_root= '/media/dataserver/workspace/blanca/training_datasets/classifier/training'
        self.drop_out= 0
        self.finetune= False
        self.gpu_ids= [0]
        self.lr= 0.00002
        self.lr_decay= 200
        self.model= 'vgg19' #'densenet' #'vgg19' # 
        self.name= 'newC'
        self.nc_input= 3
        self.nepochs= 200
        self.not_save= False
        self.nworkers= 16
        self.optimizer= 'adam'
        self.phase= 'train'
        self.phases= ['train', 'val']
        self.target_size= (224, 224)
        
opt= Options()

expr_dir = os.path.join(opt.checkpoint_dir, opt.name)
if not os.path.exists(expr_dir): os.makedirs(expr_dir)

In [3]:
import torch
from torchvision import models

num_labels = 2

if opt.model == 'densenet':
        model = models.densenet201(pretrained=True)
        model.classifier = torch.nn.Linear(model.classifier.in_features, num_labels)
        for param in model.features.parameters(): param.requires_grad = False

if opt.model == 'vgg19':
        model = models.vgg19_bn(pretrained=True)
        model.classifier._modules['6'] = torch.nn.Linear(model.classifier._modules['6'].in_features, 2)
        for param in model.features.parameters(): param.requires_grad = True
            
if opt.model == 'resnet':
        model = models.resnet18(pretrained=True)
        model.fc = torch.nn.Linear(model.fc.in_features, 2)
        for param in model.parameters(): param.requires_grad = True

# model.classifier.
optimizer = torch.optim.Adam(model.parameters(), lr=opt.lr, betas=(opt.beta1, 0.999))
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=opt.lr_decay, gamma=0.1)            
if torch.cuda.is_available():
        model = torch.nn.DataParallel(model, device_ids=list(range(torch.cuda.device_count()))).cuda()


#         model = networks.DenseNetMulti(nchannels=opt.nc_input, drop_rate=opt.drop_out)
#         model.initialize(opt)
        
# if opt.model == 'resnet':
#         model = networks.resnet18_bn(pretrained=True)
#         opt.nc_input = 3

In [4]:
from importlib import reload
import sys, os
sys.path.append(os.getcwd())
from models import fitting_classifier
# from models.fitting_classifier.train import train_model

# Setting the data loaders
from models.fitting_classifier.ClassDataset import *
from models.fitting_classifier.data_loading import *
datasets, dataloaders = set_dataloaders(opt)
datasets_sizes = {x: len(datasets[x]) for x in opt.phases}

criterion = torch.nn.BCEWithLogitsLoss()

Setting /media/dataserver/workspace/blanca/training_datasets/classifier/training/train/good_fit as 1: 
shuffling...done
Setting /media/dataserver/workspace/blanca/training_datasets/classifier/training/train/bad_fit as 0: 
shuffling...done
shuffling...done
Setting /media/dataserver/workspace/blanca/training_datasets/classifier/training/val/good_fit as 1: 
shuffling...done
Setting /media/dataserver/workspace/blanca/training_datasets/classifier/training/val/bad_fit as 0: 
shuffling...done
shuffling...done
dataset [train] was created with 2054 images
dataset [val] was created with 128 images


In [5]:
from torch.autograd import Variable
import time

def on_hot_tensor(x):
    x_ = preds.view(len(x), 1)
    if torch.cuda.is_available(): 
        x_onehot = torch.cuda.FloatTensor(len(x_), 2).zero_().scatter_(1, x_, 1)
    else: x_onehot = torch.FloatTensor(len(x_), 2).zero_().scatter_(1, x_, 1)     
    return x_onehot  

def train_model(model, optimizer, scheduler, dataloaders, criterion, opt, datasets_sizes):
    
#     for param in model.parameters(): 
#             param.requires_grad = True
    
#     if opt.finetune: 
#             for param in model.parameters(): param.requires_grad = False
#             for param in model.module.classifier.parameters(): param.requires_grad = True
          
    train_hist = {phase: {'loss': [], 'acc': []} for phase in opt.phases}
    
    since = time.time()

    best_model_wts = model.state_dict()
    best_loss = 1.0
    best_loss_acc = 0.0

    for epoch in range(opt.nepochs):
        print('\nEpoch {}/{}'.format(epoch, opt.nepochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in opt.phases:
            if phase == 'train':
                scheduler.step()
                print('lr: ', scheduler.get_lr()[0])
                model.train(True)  # Set model to training mode
            else:
                model.train(False)  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            # print("We are in epoch %d phase %s" %(epoch, phase))
            
            # Iterate over data.
            for data in dataloaders[phase]:
                # get the inputs
                inputs, labels = data

                # ON_HOT LABELS IF CRITERION REQUIRES IT
                # print (labels, type(labels))
                # print(criterion)
                if isinstance(criterion, torch.nn.modules.loss.BCEWithLogitsLoss):
                        y = labels.view(len(labels), 1).long()
                        y_onehot = torch.FloatTensor(len(y), 2).zero_().scatter_(1, y, 1)
                        labels = y_onehot
                        
                # wrap them in Variable
                if torch.cuda.is_available():
                    inputs = Variable(inputs.cuda())
                    labels = Variable(labels.cuda())
                else:
                    inputs, labels = Variable(inputs), Variable(labels)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                outputs = model(inputs)
                _, preds = torch.max(outputs.data, 1)
                
                if isinstance(criterion, torch.nn.modules.loss.BCEWithLogitsLoss):
                    y = preds.view(len(preds), 1)
                    if torch.cuda.is_available():
                        y_onehot = torch.cuda.FloatTensor(len(y), 2).zero_().scatter_(1, y, 1)
                    else: y_onehot = torch.FloatTensor(len(y), 2).zero_().scatter_(1, y, 1)     
                    preds = y_onehot
                
                # print(preds, labels)
                loss = criterion(outputs, labels)
                acc = torch.sum(preds == labels.data) / (opt.batch_size * preds.shape[1])

                # print("Batch loss/accuracy", loss, acc)
                
                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                # statistics
                running_loss += loss.data[0]
                running_corrects += torch.sum(preds == labels.data) / preds.shape[1]
                
                train_hist[phase]['loss'].append(loss.data[0])
                train_hist[phase]['acc'].append(acc)
                
            epoch_loss = running_loss / datasets_sizes[phase]
            epoch_acc = running_corrects / datasets_sizes[phase]
            
            print('{} -- Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            if phase == 'val' and epoch_loss < best_loss:
                best_loss = epoch_loss
                best_loss_acc = epoch_acc
                best_model_wts = model.state_dict()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val loss: {:4f}'.format(best_loss))
    print('Best val acc.: {:4f}'.format(best_loss_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    
    if not opt.not_save:
            print('Saving...')
            output_dir = os.path.join(opt.checkpoint_dir, opt.name)
            filename = '%s_net_loss_%f_acc_%f_%d.pth' % ('C', best_loss, best_loss_acc, int(time.time()))
            output_path = os.path.join(output_dir, filename)
            
            def try_saving(model, output_path=output_path):
                success = 0
                while success==0:
                    try:
                        torch.save(model.module.cpu().state_dict(), output_path)
                        success=1
                        print(success)
                    except: 
                        torch.save(model.module.cpu().state_dict(), output_path)
            
            try: try_saving(model, output_path)
            except: try_saving(model, output_path)
                
            if torch.cuda.is_available(): model.cuda()
    
            # save history of losses to the disk
            file_name = os.path.join(output_dir, filename + '_losses.txt')
            with open(file_name, 'wt') as opt_file:
                for k, v in train_hist.items(): 
                        opt_file.write('%s: %s\n' % (str(k), str(v)))
    
    return model

In [6]:
model = train_model(model, optimizer, scheduler, dataloaders, criterion, opt, datasets_sizes)


Epoch 0/199
----------
lr:  2e-05
train -- Loss: 0.0206 Acc: 0.6256
val -- Loss: 0.0191 Acc: 0.6562

Epoch 1/199
----------
lr:  2e-05
train -- Loss: 0.0141 Acc: 0.8262
val -- Loss: 0.0181 Acc: 0.6797

Epoch 2/199
----------
lr:  2e-05
train -- Loss: 0.0094 Acc: 0.9192
val -- Loss: 0.0191 Acc: 0.6406

Epoch 3/199
----------
lr:  2e-05
train -- Loss: 0.0053 Acc: 0.9781
val -- Loss: 0.0209 Acc: 0.6797

Epoch 4/199
----------
lr:  2e-05
train -- Loss: 0.0030 Acc: 0.9912
val -- Loss: 0.0236 Acc: 0.6406

Epoch 5/199
----------
lr:  2e-05
train -- Loss: 0.0018 Acc: 0.9981
val -- Loss: 0.0254 Acc: 0.6328

Epoch 6/199
----------
lr:  2e-05
train -- Loss: 0.0015 Acc: 0.9981
val -- Loss: 0.0262 Acc: 0.6328

Epoch 7/199
----------
lr:  2e-05
train -- Loss: 0.0009 Acc: 0.9995
val -- Loss: 0.0262 Acc: 0.6562

Epoch 8/199
----------
lr:  2e-05
train -- Loss: 0.0008 Acc: 0.9995
val -- Loss: 0.0278 Acc: 0.6719

Epoch 9/199
----------
lr:  2e-05
train -- Loss: 0.0013 Acc: 0.9971
val -- Loss: 0.0273 Ac

val -- Loss: 0.0392 Acc: 0.6562

Epoch 81/199
----------
lr:  2e-05
train -- Loss: 0.0001 Acc: 0.9995
val -- Loss: 0.0381 Acc: 0.6562

Epoch 82/199
----------
lr:  2e-05
train -- Loss: 0.0001 Acc: 0.9985
val -- Loss: 0.0436 Acc: 0.6719

Epoch 83/199
----------
lr:  2e-05
train -- Loss: 0.0001 Acc: 1.0000
val -- Loss: 0.0402 Acc: 0.6328

Epoch 84/199
----------
lr:  2e-05
train -- Loss: 0.0000 Acc: 1.0000
val -- Loss: 0.0406 Acc: 0.6328

Epoch 85/199
----------
lr:  2e-05
train -- Loss: 0.0000 Acc: 1.0000
val -- Loss: 0.0409 Acc: 0.6406

Epoch 86/199
----------
lr:  2e-05
train -- Loss: 0.0000 Acc: 1.0000
val -- Loss: 0.0401 Acc: 0.6328

Epoch 87/199
----------
lr:  2e-05
train -- Loss: 0.0000 Acc: 1.0000
val -- Loss: 0.0459 Acc: 0.6562

Epoch 88/199
----------
lr:  2e-05
train -- Loss: 0.0000 Acc: 1.0000
val -- Loss: 0.0450 Acc: 0.6562

Epoch 89/199
----------
lr:  2e-05
train -- Loss: 0.0000 Acc: 1.0000
val -- Loss: 0.0450 Acc: 0.6484

Epoch 90/199
----------
lr:  2e-05
train -- Loss:

val -- Loss: 0.0484 Acc: 0.6484

Epoch 161/199
----------
lr:  2e-05
train -- Loss: 0.0001 Acc: 0.9995
val -- Loss: 0.0516 Acc: 0.6484

Epoch 162/199
----------
lr:  2e-05
train -- Loss: 0.0001 Acc: 0.9995
val -- Loss: 0.0488 Acc: 0.6641

Epoch 163/199
----------
lr:  2e-05
train -- Loss: 0.0001 Acc: 0.9995
val -- Loss: 0.0477 Acc: 0.6641

Epoch 164/199
----------
lr:  2e-05
train -- Loss: 0.0000 Acc: 1.0000
val -- Loss: 0.0469 Acc: 0.7031

Epoch 165/199
----------
lr:  2e-05
train -- Loss: 0.0000 Acc: 1.0000
val -- Loss: 0.0457 Acc: 0.7188

Epoch 166/199
----------
lr:  2e-05
train -- Loss: 0.0002 Acc: 0.9995
val -- Loss: 0.0446 Acc: 0.7109

Epoch 167/199
----------
lr:  2e-05
train -- Loss: 0.0002 Acc: 0.9976
val -- Loss: 0.0493 Acc: 0.6797

Epoch 168/199
----------
lr:  2e-05
train -- Loss: 0.0001 Acc: 0.9995
val -- Loss: 0.0467 Acc: 0.6719

Epoch 169/199
----------
lr:  2e-05
train -- Loss: 0.0003 Acc: 0.9976
val -- Loss: 0.0451 Acc: 0.6875

Epoch 170/199
----------
lr:  2e-05
trai

In [25]:
# import cv2
# import numpy as np
# import matplotlib.pyplot as plt
# %matplotlib inline 

# def show(img):
#     plt.imshow(img, cmap="gray")
#     plt.show()

# def ocr(img):
#     # Tesseract mode settings:
#     #   Page Segmentation mode (PSmode) = 3 (defualt = 3)
#     #   OCR Enginer Mode (OEM) = 3 (defualt = 3)
#     tesser = cv2.text.OCRTesseract_create('C:/Program Files/Tesseract 4.0.0/tessdata/','eng','0123456789',3,3)
#     retval = tesser.run(img, 0) # return string type
#     print ('OCR Output: ' + retval)

# # Directly feed image to Tesseact
# img = cv2.imread('./imagesStackoverflow/SmallDigits-x4.png')
# ocr(im1)

# # Load image as gray scale 
# img = cv2.imread('./imagesStackoverflow/SmallDigits-x4.png', 0);
# show(im1)
# ocr(im1)

# # Enhance image and get same positive result
# ret, thresh = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY) 
# kernel = np.ones((3,3), np.uint8)
# img = cv2.erode(thresh, kernel, iterations = 1)
# show(img)
# ocr(img)

error: /home/travis/miniconda/conda-bld/conda_1486587069159/work/opencv-3.1.0/build/opencv_contrib/modules/text/src/ocr_tesseract.cpp:187: error: (-215) (image.type() == CV_8UC1) || (image.type() == CV_8UC3) in function run


In [10]:
res = models.resnet18(pretrained=True)
res.fc

Linear(in_features=512, out_features=1000, bias=True)

In [29]:
for i, j in res._modules.items():
    print(i)

conv1
bn1
relu
maxpool
layer1
layer2
layer3
layer4
avgpool
fc
